# Producer Notebook - Streaming Weather to Kafka

## Prerequisites

In [ ]:
%pip install confluent_kafka requests

## Configuration

In [ ]:
# Import necessary libraries
import time
import json
import requests
from confluent_kafka import Producer

# Configuration Parameters

# OpenWeatherMap API Configuration
OPENWEATHERMAP_API_KEY = 'YOUR_OPENWEATHERMAP_API_KEY'  # Replace with your actual API key
CITY_NAME = 'London'                                    # Replace with your desired city
COUNTRY_CODE = 'UK'                                     # Replace with your country code
UNITS = 'metric'                                        # 'metric' or 'imperial'

# Kafka Configuration
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'             # Kafka broker address
KAFKA_TOPIC = 'weather'                                 # Kafka topic name

# OpenWeatherMap API Endpoint
OWM_ENDPOINT = 'https://api.openweathermap.org/data/2.5/weather'

# Fetch Interval (in seconds)
FETCH_INTERVAL = 600  # 600 seconds = 10 minutes

## Setting Up the Kafka Producer

In [ ]:
def delivery_callback(err, msg):
    """
    Callback function called once for each message produced to indicate delivery result.
    
    :param err: Error information, if any
    :param msg: The message produced
    """
    if err:
        print(f"[ERROR] Message delivery failed: {err}")
    else:
        print(f"[SUCCESS] Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

In [ ]:
# Kafka Producer Configuration
producer_config = {
    'bootstrap.servers': KAFKA_BOOTSTRAP_SERVERS,
    'acks': 'all',           # Ensure all replicas acknowledge
    'retries': 5,            # Retry up to 5 times on failure
    'linger.ms': 10,         # Wait up to 10ms to batch messages
    'batch.size': 32768      # Batch size in bytes (32KB)
}

# Create Producer instance
producer = Producer(producer_config)

## Fetching Weather Data

In [ ]:
def get_weather_data(api_key, city, country, units='metric'):
    """
    Fetches weather data from OpenWeatherMap API for the specified location.
    
    :param api_key: API key for OpenWeatherMap
    :param city: City name
    :param country: Country code (e.g., 'UK')
    :param units: Units of measurement ('metric' or 'imperial')
    :return: Dictionary containing weather data or None if failed
    """
    params = {
        'q': f'{city},{country}',
        'appid': api_key,
        'units': units
    }
    try:
        response = requests.get(OWM_ENDPOINT, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Extract desired weather stats
        weather = {
            'city': data.get('name'),
            'country': data.get('sys', {}).get('country'),
            'temperature': data.get('main', {}).get('temp'),
            'humidity': data.get('main', {}).get('humidity'),
            'weather_description': data.get('weather', [{}])[0].get('description'),
            'timestamp': data.get('dt')  # Unix timestamp
        }
        return weather
    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Failed to fetch weather data: {e}")
        return None

## Producing Messages to Kafka

In [ ]:
def produce_weather_data(producer, topic, weather_data):
    """
    Produces weather data to the specified Kafka topic.
    
    :param producer: Kafka Producer instance
    :param topic: Kafka topic name
    :param weather_data: Dictionary containing weather data
    """
    try:
        # Serialize weather data to JSON string
        weather_json = json.dumps(weather_data)
        
        # Optional: Use city name as the message key for partitioning
        key = weather_data['city'].encode('utf-8')
        
        # Produce message to Kafka
        producer.produce(
            topic=topic,
            key=key,
            value=weather_json,
            callback=delivery_callback
        )
        
        # Trigger delivery report callbacks
        producer.poll(0)
    except Exception as e:
        print(f"[ERROR] Failed to produce message: {e}")

## Main Execution Loop

In [ ]:
def main():
    print(f"Starting Kafka producer for weather data: {CITY_NAME}, {COUNTRY_CODE}")
    print(f"Producing to topic '{KAFKA_TOPIC}' every {FETCH_INTERVAL} seconds.\n")
    
    try:
        while True:
            # Fetch weather data
            weather_data = get_weather_data(
                OPENWEATHERMAP_API_KEY,
                CITY_NAME,
                COUNTRY_CODE,
                UNITS
            )
            
            if weather_data:
                # Produce weather data to Kafka
                produce_weather_data(producer, KAFKA_TOPIC, weather_data)
            
            # Wait for the specified interval before next fetch
            time.sleep(FETCH_INTERVAL)
    
    except KeyboardInterrupt:
        print("\n[INFO] Producer interrupted by user. Flushing messages...")
    
    except Exception as e:
        print(f"[ERROR] An unexpected error occurred: {e}")
    
    finally:
        # Flush any remaining messages
        producer.flush()
        print("[INFO] Producer has been shut down.")

# Entry Point

In [ ]:
if __name__ == '__main__':
    main()